In [3]:
import fnmatch
import os
import os.path
import re

def walknroll(fun, includes=['*.html', '_posts*'], excludes = ['*all_static*', '*_includes*', '*_layouts*', '*_drafts*']):
    # transform glob patterns to regular expressions
    includes = r'|'.join([fnmatch.translate(x) for x in includes])
    excludes = r'|'.join([fnmatch.translate(x) for x in excludes]) or r'$.'

    for root, dirs, files in os.walk('..'):
        # exclude/include files
        files = [os.path.join(root, f) for f in files]
        files = [f for f in files if not re.match(excludes, f)]
        files = [f for f in files if re.match(includes, f)]

        for fname in files:
            fun(fname)

In [4]:
from collections import OrderedDict as OD

class NotPost(Exception):
    pass

class Post:
    def __init__(self, fn):
        self.fn = fn
        content = open(fn).read()
        if '---' not in content:
            raise NotPost
        self.fm = content.split('---')[1] # front matter
        self.body = content.split('---')[2]
        self.fm_vars = OD((l.split(':')[0], l.split(': ')[1]) for l in self.fm.split('\n') if ': ' in l)
    
    def head(self):
        return ('---\n' +
            '\n'.join([k+': '+v for k, v in self.fm_vars.iteritems()]) +
            '\n---')
    
    def _content(self):
        return self.head() + self.body

    def __str__(self):
        return self._content()
    
    def __repr__(self):
        return 'Post(\'{}\')'.format(self.fn)
    
    def save(self):
        with open(self.fn, 'w') as f:
            f.write(self._content())

# old scripts - examples

### re-categorize the javascript index pages

In [21]:
categories = {
    '3D Cluster Graph': '3d_charts',
    '3D Line Plots': '3d_charts',
    '3D Scatter Plots': '3d_charts', 
    '3D Surface Plots': '3d_charts',
    'Read CSV Data from an Ajax Call': 'tutorials',
    'Text and Annotations': 'layout_opt',
    'Filled Area Plots': 'basic',
    'Axes': 'layout_opt',
    'Bar Charts': 'basic',
    'Box Plots': 'basic',
    'Bubble Charts': 'basic',
    'Bubble Maps': 'maps',
    'Button Events': 'chart_events',
    'Click Events': 'chart_events',
    'Dropdown Events': 'chart_events',
    'Hover Events': 'chart_events',
    'Slider Events': 'chart_events',
    'Zoom Events': 'event',
    'Candlestick Charts': 'finance',
    'Choropleth Maps': 'maps', 
    'Heatmap and Contour Colorscales': 'scientific',
    'Configuration Options': 'reference',
    'Contour Plots': 'scientific',
    '2d Density Plots': 'statistical',
    'Error Bars': 'statistical',
    'Text and Font Styling': 'layout_opt',
    'Heatmaps': 'scientific',
    'Histograms': 'statistical',
    '2D Histograms': 'statistical',
    'Horizontal Bar Charts': 'basic',
    'Hover Text and Formatting': 'layout_opt',
    'Inset Plots': 'basic',
    'Setting the Title, Legend Entries, and Axis Titles': 'layout_opt',
    'LaTeX': 'layout_opt',
    'Legends': 'layout_opt',
    'Line Charts': 'basic',
    'Scatter Plots': 'basic',
    'Lines on Maps': 'maps',
    'Log Plots': 'basic',
    'Multiple Chart Types': 'basic',
    'Multiple Axes': 'basic',
    'OHLC Charts': 'finance',
    'Pie Charts': 'basic',
    'Polar Charts': 'scientific',
    'Remove Trace from Plot': 'reference',
    'Responsive / Fluid Layout': 'tutorials',
    'Ribbon Plots': '3d_charts',
    'Scatter Plots on Maps': 'maps',
    'Shapes': 'basic',
    'Setting Graph Size': 'layout_opt',
    'Subplots': 'basic',
    'Time Series': 'basic',
    'Wind Rose Charts': 'scientific'
}

In [22]:
def recategorize_posts(fn):
    try:
        p = Post(fn)
    except NotPost:
        pass
    except Exception as e:
        print fn
        raise e
    else:
        if p.fm_vars.get('language', '') == 'plotly_js' and p.fm_vars.get('name', '') in categories:
            p.fm_vars['display_as'] = categories[p.fm_vars.get('name')]
            p.save()

walknroll(recategorize_posts)

### change urls of all the javascript pages

In [ ]:
def process_posts(fn):
    try:
        p = Post(fn)
    except NotPost:
        pass
    except Exception as e:
        print fn
        raise e
    else:
        if 'permalink' in p.fm_vars and 'javascript-graphing' in p.fm_vars['permalink']:
            p.fm_vars['redirect_from'] = p.fm_vars['permalink']
            p.fm_vars['permalink'] = p.fm_vars['permalink'].replace('javascript-graphing-library', 'javascript')
            p.save()

### update graph_objs

In [169]:
import plotly.graph_objs as go

trace_objs = ['Area', 'Bar', 'Box', 'Choropleth', 'Figure', 'Heatmap', 'Histogram', 'Histogram2d',
              'Histogram2dcontour', 'Layout', 'Mesh3d', 'Pie', 'Scatter', 'Scatter3d', 'Scattergeo',
              'Surface']

old_objs = ['AngularAxis', 'Annotation', 'Annotations', 'ColorBar', 'Contour', 'Contours',
            'ErrorX', 'ErrorY', 'ErrorZ', 'Figure', 'Font', 'Legend', 'Line', 'Margin',
            'Marker', 'RadialAxis', 'Scene', 'Stream', 'XAxis', 'XBins', 'YAxis', 'YBins', 'ZAxis']

class FailedExec(Exception):
    pass

def update_post(fn):
    try:
        p = Post(fn)
    except NotPost:
        pass
    except Exception as e:
        print 'wtf not post?'.format(fn)

    else:
        if p.fm_vars.get('language', '') == 'python' and p.fm_vars.get('page_type', '') != 'example_index':
            print 'processing {}'.format(fn)
            new_post = p.body
            # make a bunch of changes to the script
            new_post = new_post.replace('from plotly.graph_objs import *', 'import plotly.graph_objs as go')
            for trace in trace_objs:
                new_post = new_post.replace(trace, 'go.{}'.format(trace))
            for deprecated in old_objs:
                new_post = new_post.replace(deprecated, 'dict')
            
            # replace Data([trace0, trace1]) with [trace0, trace1]
            r = r'Data\(([a-zA-Z =([\d,\])]*)\)'
            reg = re.compile(r)            
            try:
                new_post = ''.join(reg.split(new_post))
            except:
                pass
            
            # exec is so sweet
            local_vars = {}
            try:
                exec(new_post, {}, local_vars)
            except:
                print 'exec failed '.format(fn)

            try:
                # hah, bam!
                p.fm_vars['plot_url'] = local_vars.get('plot_url', local_vars.get('url'))
            except:
                print 'plot_url not defined '.format(fn)
            
            p.body = new_post
            
            p.save()

In [ ]:
%pdb
walknroll(update_post)

Automatic pdb calling has been turned ON
processing ../_posts/2015-04-05-python-index.html
exec failed 
processing ../_posts/2015-05-25-ipython-notebooks_gallery.html
exec failed 
processing ../_posts/2015-05-25-ipython-notebooks_index.html
exec failed 
processing ../_posts/2015-09-06-python-reference.html
exec failed 
wtf not post?
wtf not post?
processing ../_posts/python/3d-clusters/2015-07-15-alpha-shape.html
processing ../_posts/python/3d-filled-line/2015-04-09-3d-filled-line.html
processing ../_posts/python/3d-line/2015-04-09-random-walk.html
exec failed 
processing ../_posts/python/3d-parametric/2015-08-21-parametric-colorscale.html
exec failed 
processing ../_posts/python/3d-parametric/2015-08-21-parametric.html
exec failed 
processing ../_posts/python/3d-ribbon/2015-07-15-ribbon.html
processing ../_posts/python/3d-scatter/2015-04-09-simple-3d-scatter.html
exec failed 
processing ../_posts/python/3d-subplot/2015-08-21-3d-subplots.html
This is the format of your plot grid:
[ (1,

In [163]:
p = Post('../_posts/2015-04-05-python-index.html')

In [165]:
p.fm_vars

OrderedDict([('title', 'Python Graphing Library, Plotly'),
             ('permalink', '/python/'),
             ('description',
              "Plotly's Python graphing library makes interactive, publication-quality graphs online. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts and bubble charts. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts and bubble charts"),
             ('layout', 'langindex'),
             ('language', 'python')])